In [130]:
import pandas as pd
import numpy as np
import xbbg
import pdblp
from xbbg import blp
from Binomial_Fixed.ficcvol import *
import Binomial_Fixed
import quandl
apikey = 'J_fXGeVW_zC6RaDeJSQv'
quandl.ApiConfig.api_key = apikey
import datetime 
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'



# Volatility Modeling

In [106]:
TICKS = [
    'OWF/CBT_FV_FV_U2023_IV',
    'OWF/CME_ES_ES_U2023_IV',
    'OWF/CBT_TY_TY_U2023_IV',
    'OWF/CBT_FF_FF_N2023_IV',
    'OWF/CME_ED_ED_U2023_IV',
]

ticksplitlabs = ['exchange','futures ticker','option ticker','monthYear']
info = pd.DataFrame(index=ticksplitlabs)
info.index.name = 'specs'
surfs = dict()
labels = dict()

In [108]:
for tick in TICKS:
    ### Download Data
    df_model = quandl.get(f'{tick}M')
    df_surf = quandl.get(f'{tick}S')

    ### Organize Labels
    finalDate = df_model.index[-1]
    tick_split = tick.split('/')[1].split('_')[0:-1]
    labels[tick] = ' '.join(tick_split)
    label = labels[tick]

    ### Data Descriptions
    info.loc[ticksplitlabs,label] = tick_split
    info.loc['option expiration',label] = (finalDate + datetime.timedelta(df_model.loc[finalDate,'DtE'])).strftime('%Y-%m-%d')
    info.loc['future expiration',label] = (finalDate + datetime.timedelta(df_model.loc[finalDate,'DtT'])).strftime('%Y-%m-%d')


    ### Organize Data
    surf = df_surf.drop(columns=['DNSvol','DtE']).join(df_model[['Future','DtE','DtT']]).rename(columns={'DtT': 'Expiration Future','DtE':'Expiration Option','Future':'Future Price'})
    surf = pd.concat([surf.iloc[:,-3:], surf.iloc[:,0:-3]],axis=1)
    surf[['Expiration Future','Expiration Option']] /= 365
    
    surfs[tick] = surf

In [117]:
from scipy.optimize import fsolve
from scipy.optimize import minimize

from pandas.tseries.offsets import Day, BDay
from datetime import date